In [ ]:
import pandas as pd
def process_umi_data(input_file, output_file):
  
    df = pd.read_excel(input_file)
   
    # Extract the list of genes from the first column (excluding column heading)
    genes = df.iloc[1:, 0]  # First column contains the list of genes, excluding row 0
   
    # Extract only the data rows (excluding header) for calculations
    data = df.iloc[1:, 1:].astype(float)  # Exclude the first row and convert to float for calculations
   
    # Calculate the sum of UMIs for each condition (columns from the second onwards)
    umi_sums = data.sum(axis=0)  # Sum UMIs for each condition
   
    # Normalize each UMI value to the sum of UMIs for that condition
    normalized_data = data.div(umi_sums, axis=1) * 100
   
    # Find the average of each gene under columns titled "native"
    native_columns = [col for col in df.columns if "native" in col.lower()]
    native_data = normalized_data[native_columns]
    native_averages = native_data.mean(axis=1)
   
    # Find the correction for native
    num_genes = len(genes)
    native_correction = (1 / num_genes) / native_averages
   
    # Multiply the native correction to normalized values for each condition * 100
    corrected_data = normalized_data.mul(native_correction, axis=0) * 100
   
    # Save all calculated values into separate sheets in an Excel file
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        
        # Save original data
        original_data = df.iloc[1:, :].reset_index(drop=True)
        original_data.columns = df.columns  # Retain original column headers
        original_data.to_excel(writer, sheet_name="Original Data", index=False)
        
        # Save normalized data
        normalized_data_with_genes = pd.DataFrame(normalized_data)
        normalized_data_with_genes.insert(0, "Gene", genes.values)
        normalized_data_with_genes.columns = ["Gene"] + [f"Normalized_{col}" for col in data.columns]
        normalized_data_with_genes.to_excel(writer, sheet_name="Normalized Data", index=False)
        
        # Save native averages
        native_averages_df = pd.DataFrame({"Gene": genes.values, "Native_Average": native_averages.values})
        native_averages_df.to_excel(writer, sheet_name="Native Averages", index=False)
        
        # Save native corrections
        native_correction_df = pd.DataFrame({"Gene": genes.values, "Native_Correction": native_correction.values})
        native_correction_df.to_excel(writer, sheet_name="Native Corrections", index=False)
        
        # Save corrected data
        corrected_data_with_genes = pd.DataFrame(corrected_data)
        corrected_data_with_genes.insert(0, "Gene", genes.values)
        corrected_data_with_genes.columns = ["Gene"] + [f"Corrected_{col}" for col in data.columns]
        corrected_data_with_genes.to_excel(writer, sheet_name="Corrected Data", index=False)
        
        # Save UMI sums as a separate row in a new sheet
        umi_sums_df = pd.DataFrame(umi_sums).transpose()
        umi_sums_df.insert(0, "Gene", "UMI_Sum")
        umi_sums_df.to_excel(writer, sheet_name="UMI Sums", index=False)
    print(f"Processed data saved to {output_file}")

# Example usage
input_file = "/Users/drk29580/Downloads/input_caspase_kras.xlsx"  # Replace with the path to your input Excel file
output_file = "/Users/drk29580/Downloads/output_caspase_kras_repeat_normalized 2.xlsx"  # Replace with the path to your desired output Excel file
process_umi_data(input_file, output_file)











Processed data saved to /Users/drk29580/Downloads/output_caspase_kras_repeat_normalized 2.xlsx


In [3]:
import pandas as pd
def process_umi_data(input_file, output_file):
   
    # Load the Excel file into a pandas DataFrame
    df = pd.read_excel(input_file)
   
    # Extract the list of genes from the first column (excluding column heading)
    genes = df.iloc[1:, 0]  # First column contains the list of genes, excluding row 0
   
    # Extract only the data rows (excluding header) for calculations
    data = df.iloc[1:, 1:].astype(float)  # Exclude the first row and convert to float for calculations
   
    # Calculate the sum of UMIs for each condition (columns from the second onwards)
    umi_sums = data.sum(axis=0)  # Sum UMIs for each condition
   
    # Normalize each UMI value to the sum of UMIs for that condition
    normalized_data = data.div(umi_sums, axis=1) * 100
   
    # Find the average of each gene under columns titled "native" after normalization
    native_columns = [col for col in df.columns if "native" in col.lower()]
    normalized_native_data = normalized_data[native_columns]  # Extract normalized native columns
    native_averages = normalized_native_data.mean(axis=1)  # Average normalized values for native columns
   
    # Find the correction for native
    num_genes = len(genes)
    native_correction = (1 / num_genes) / native_averages
   
    # Multiply the native correction to normalized values for each condition * 100
    corrected_data = normalized_data.mul(native_correction, axis=0) * 100
   
    # Calculate correction values for `unsorted_<suffix>` columns
    unsorted_columns = [col for col in df.columns if col.startswith("unsorted_")]
    unsorted_corrections = {}
    for col in unsorted_columns:
        unsorted_corrections[col] = (1 / num_genes) / normalized_data[col]
   
    # Calculate corrected values for columns like `live_<suffix>`, `dead_<suffix>`, etc.
    corrected_unsorted_related_data = pd.DataFrame(index=normalized_data.index)
    for col in normalized_data.columns:
   
        # Extract the suffix from the column name (e.g., "5nm" from "live_5nm")
        if "_" in col:
            suffix = col.split("_")[-1]
            corresponding_unsorted_col = f"unsorted_{suffix}"
            # Check if the corresponding unsorted column exists
            if corresponding_unsorted_col in unsorted_corrections:
                corrected_unsorted_related_data[col] = normalized_data[col] * unsorted_corrections[corresponding_unsorted_col] * 100
   
    # Save all calculated values into separate sheets in an Excel file
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        # Save original data
        original_data = df.iloc[1:, :].reset_index(drop=True)
        original_data.columns = df.columns  # Retain original column headers
        original_data.to_excel(writer, sheet_name="Original Data", index=False)
        
        # Save normalized data
        normalized_data_with_genes = pd.DataFrame(normalized_data)
        normalized_data_with_genes.insert(0, "Gene", genes.values)
        normalized_data_with_genes.columns = ["Gene"] + [f"Normalized_{col}" for col in data.columns]
        normalized_data_with_genes.to_excel(writer, sheet_name="Normalized Data", index=False)
        
        # Save native averages
        native_averages_df = pd.DataFrame({"Gene": genes.values, "Native_Average": native_averages.values})
        native_averages_df.to_excel(writer, sheet_name="Native Averages", index=False)
        
        # Save native corrections
        native_correction_df = pd.DataFrame({"Gene": genes.values, "Native_Correction": native_correction.values})
        native_correction_df.to_excel(writer, sheet_name="Native Corrections", index=False)
        
        # Save corrected data
        corrected_data_with_genes = pd.DataFrame(corrected_data)
        corrected_data_with_genes.insert(0, "Gene", genes.values)
        corrected_data_with_genes.columns = ["Gene"] + [f"Corrected_{col}" for col in data.columns]
        corrected_data_with_genes.to_excel(writer, sheet_name="Corrected Data", index=False)
        
        # Save unsorted corrections
        unsorted_corrections_df = pd.DataFrame(unsorted_corrections).transpose()
        unsorted_corrections_df.columns = genes.values
        unsorted_corrections_df.insert(0, "Unsorted Column", unsorted_corrections.keys())
        unsorted_corrections_df.to_excel(writer, sheet_name="Unsorted Corrections", index=False)
        
        # Save corrected unsorted-related data
        corrected_unsorted_related_with_genes = corrected_unsorted_related_data.copy()
        corrected_unsorted_related_with_genes.insert(0, "Gene", genes.values)
        corrected_unsorted_related_with_genes.to_excel(writer, sheet_name="Unsorted Corrected Data", index=False)
        
        # Save UMI sums as a separate row in a new sheet
        umi_sums_df = pd.DataFrame(umi_sums).transpose()
        umi_sums_df.insert(0, "Gene", "UMI_Sum")
        umi_sums_df.to_excel(writer, sheet_name="UMI Sums", index=False)
    print(f"Processed data saved to {output_file}")

# Example usage
input_file = "/Users/drk29580/Downloads/input_caspase_kras.xlsx"  # Replace with the path to your input Excel file
output_file = "/Users/drk29580/Downloads/output_caspase_kras_unsorted.xlsx"  # Replace with the path to your desired output Excel file
process_umi_data(input_file, output_file)








Processed data saved to /Users/drk29580/Downloads/output_caspase_kras_unsorted.xlsx


In [ ]:
import pandas as pd
def process_umi_data(input_file, output_file):
   
    # Load the Excel file into a pandas DataFrame
    df = pd.read_excel(input_file)
   
    # Extract the list of genes from the first column (excluding column heading)
    genes = df.iloc[1:, 0]  # First column contains the list of genes, excluding row 0
   
    # Extract only the data rows (excluding header) for calculations
    data = df.iloc[1:, 1:].astype(float)  # Exclude the first row and convert to float for calculations
   
    # Calculate the sum of UMIs for each condition (columns from the second onwards)
    umi_sums = data.sum(axis=0)  # Sum UMIs for each condition
   
    # Normalize each UMI value to the sum of UMIs for that condition
    normalized_data = data.div(umi_sums, axis=1) * 100
   
    # Find the average of each gene under columns titled "native" after normalization
    native_columns = [col for col in df.columns if "native" in col.lower()]
    normalized_native_data = normalized_data[native_columns]  # Extract normalized native columns
    native_averages = normalized_native_data.mean(axis=1)  # Average normalized values for native columns
   
    # Find the correction for native
    num_genes = len(genes)
    native_correction = (1 / num_genes) / native_averages
   
    # Multiply the native correction to normalized values for each condition * 100
    corrected_data = normalized_data.mul(native_correction, axis=0) * 100
   
    # Calculate correction values for `unsorted_<suffix>` columns
    unsorted_columns = [col for col in df.columns if col.startswith("unsorted_")]
    unsorted_corrections = {}
    for col in unsorted_columns:
        unsorted_corrections[col] = (1 / num_genes) / corrected_data[col]
   
    # Calculate corrected values for columns like `live_<suffix>`, `dead_<suffix>`, etc.
    corrected_unsorted_related_data = pd.DataFrame(index=corrected_data.index)
    for col in corrected_data.columns:
   
        # Extract the suffix from the column name (e.g., "5nm" from "live_5nm")
        if "_" in col:
            suffix = col.split("_")[-1]
            corresponding_unsorted_col = f"unsorted_{suffix}"
            # Check if the corresponding unsorted column exists
            if corresponding_unsorted_col in unsorted_corrections:
                corrected_unsorted_related_data[col] = corrected_data[col] * unsorted_corrections[corresponding_unsorted_col] * 100
   
    # Save all calculated values into separate sheets in an Excel file
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        # Save original data
        original_data = df.iloc[1:, :].reset_index(drop=True)
        original_data.columns = df.columns  # Retain original column headers
        original_data.to_excel(writer, sheet_name="Original Data", index=False)
        
        # Save normalized data
        normalized_data_with_genes = pd.DataFrame(normalized_data)
        normalized_data_with_genes.insert(0, "Gene", genes.values)
        normalized_data_with_genes.columns = ["Gene"] + [f"Normalized_{col}" for col in data.columns]
        normalized_data_with_genes.to_excel(writer, sheet_name="Normalized Data", index=False)
        
        # Save native averages
        native_averages_df = pd.DataFrame({"Gene": genes.values, "Native_Average": native_averages.values})
        native_averages_df.to_excel(writer, sheet_name="Native Averages", index=False)
        
        # Save native corrections
        native_correction_df = pd.DataFrame({"Gene": genes.values, "Native_Correction": native_correction.values})
        native_correction_df.to_excel(writer, sheet_name="Native Corrections", index=False)
        
        # Save corrected data
        corrected_data_with_genes = pd.DataFrame(corrected_data)
        corrected_data_with_genes.insert(0, "Gene", genes.values)
        corrected_data_with_genes.columns = ["Gene"] + [f"Corrected_{col}" for col in data.columns]
        corrected_data_with_genes.to_excel(writer, sheet_name="Corrected Data", index=False)
        
        # Save unsorted corrections
        unsorted_corrections_df = pd.DataFrame(unsorted_corrections).transpose()
        unsorted_corrections_df.columns = genes.values
        unsorted_corrections_df.insert(0, "Unsorted Column", unsorted_corrections.keys())
        unsorted_corrections_df.to_excel(writer, sheet_name="Unsorted Corrections", index=False)
        
        # Save corrected unsorted-related data
        corrected_unsorted_related_with_genes = corrected_unsorted_related_data.copy()
        corrected_unsorted_related_with_genes.insert(0, "Gene", genes.values)
        corrected_unsorted_related_with_genes.to_excel(writer, sheet_name="Unsorted Corrected Data", index=False)
        
        # Save UMI sums as a separate row in a new sheet
        umi_sums_df = pd.DataFrame(umi_sums).transpose()
        umi_sums_df.insert(0, "Gene", "UMI_Sum")
        umi_sums_df.to_excel(writer, sheet_name="UMI Sums", index=False)
    print(f"Processed data saved to {output_file}")

# Example usage
input_file = "/Users/drk29580/Downloads/Sample input test.xlsx"  # Replace with the path to your input Excel file
output_file = "/Users/drk29580/Downloads/Sample output2 test.xlsx"  # Replace with the path to your desired output Excel file
process_umi_data(input_file, output_file)






